In [1]:
import os
import zipfile
import gdown
from dataclasses import dataclass
from pathlib import Path

from CNNClassifier.constants import *
from CNNClassifier import logger
from CNNClassifier.utils.common import read_yaml, create_directories, get_size

In [2]:
%pwd

'c:\\Users\\Swaraj\\Desktop\\Learning\\E2ECVProject\\E2ECancerClassification\\research'

In [3]:
os.chdir("../")

In [4]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_date_file: Path
    unzip_dir: Path

In [11]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath: Path = CONFIG_FILE_PATH,
            params_filepath: Path = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_date_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [16]:
class DataIngestion:

    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def download_file(self) -> str:
        """Fetch data from the URL"""
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_date_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_date_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-05 16:57:05,543: INFO: common: yaml file: config\config.yaml loaded successfully!]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2024-02-05 16:57:05,546: INFO: common: yaml file: params.yaml loaded successfully!]
{'key': 'value'}
[2024-02-05 16:57:05,548: INFO: common: Created directory at : artifacts]
[2024-02-05 16:57:05,549: INFO: common: Created directory at : artifacts/data_ingestion]
[2024-02-05 16:57:05,552: INFO: 203519484: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=f15228d1-1d2d-4246-89b7-f52201b3f35d
To: c:\Users\Swaraj\Desktop\Learning\E2ECVProject\E2ECancerClassification\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:14<00:00, 3.37MB/s]

[2024-02-05 16:57:21,216: INFO: 203519484: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
